In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

from nltk import tokenize
from sklearn.model_selection import train_test_split
from nltk.corpus import subjectivity
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.util import *
from nltk.classify import NaiveBayesClassifier

In [2]:
df = pd.read_csv("twitter_data.csv",encoding="ISO-8859-1",names=["target", "ids", "date", "flag", "user", "text"])

In [3]:
df = df.dropna()
del df["flag"]
del df["user"]
del df["date"]
if (len(df["target"].unique()) == 1):
    del df["target"]
else:
    print(df["target"].unique())
df.head()

[0 4]


,target,ids,text
0,0,1467810369,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,is upset that he can't update his Facebook by ...
2,0,1467810917,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,my whole body feels itchy and like its on fire
4,0,1467811193,"@nationwideclass no, it's not behaving at all...."


In [4]:
df = df.replace(4, 1)
df.target.unique()
#df = df.sample(frac = .05)


array([0, 1])

In [5]:
df["split_text"] = df["text"].str.split()

In [6]:
df["final_text"] = list(zip(df["split_text"], df["target"]))

In [7]:

from nltk.corpus import stopwords
stops = set(stopwords.words('english'))

new_list = []
for sentence in df["final_text"].tolist():
    temp_list = []
    
    for word in sentence[0]:
        if word not in stops:
            if word[0] != '@':
                if ":" not in word:
                    if "/" not in word:
                        temp_list.append(word)
                        
            
    new_list.append((temp_list,sentence[1]))
    
    

    


In [8]:
df["final_text"] = new_list
df.head()

,target,ids,text,split_text,final_text
0,0,1467810369,"@switchfoot http://twitpic.com/2y1zl - Awww, t...","[@switchfoot, http://twitpic.com/2y1zl, -, Aww...","([-, Awww,, that's, bummer., You, shoulda, got..."
1,0,1467810672,is upset that he can't update his Facebook by ...,"[is, upset, that, he, can't, update, his, Face...","([upset, can't, update, Facebook, texting, it...."
2,0,1467810917,@Kenichan I dived many times for the ball. Man...,"[@Kenichan, I, dived, many, times, for, the, b...","([I, dived, many, times, ball., Managed, save,..."
3,0,1467811184,my whole body feels itchy and like its on fire,"[my, whole, body, feels, itchy, and, like, its...","([whole, body, feels, itchy, like, fire], 0)"
4,0,1467811193,"@nationwideclass no, it's not behaving at all....","[@nationwideclass, no,, it's, not, behaving, a...","([no,, behaving, all., i'm, mad., here?, I, ca..."


In [9]:
# Creating a dataframe with 80%

#df = df.sample(frac = .02)

# values of original dataframe
train_df = df.sample(frac = 0.8)
 
# Creating dataframe with
# rest of the 20% values
test_df = df.drop(train_df.index)

#We have train and test, now we need to...
# Today's goal: Get a model, train it, test it

In [10]:
sentim_analyzer = SentimentAnalyzer() # Set up model

In [11]:
all_words_neg = sentim_analyzer.all_words([mark_negation(doc) for doc in train_df["final_text"].tolist()])

In [12]:
unigram_feats = sentim_analyzer.unigram_word_feats(all_words_neg, min_freq=50)

In [13]:
sentim_analyzer.add_feat_extractor(extract_unigram_feats, unigrams=unigram_feats)
print(len(unigram_feats))

16103


In [14]:
training_set = sentim_analyzer.apply_features(train_df["final_text"].tolist())

In [15]:
test_set = sentim_analyzer.apply_features(test_df["final_text"].tolist())

In [16]:
trainer = NaiveBayesClassifier.train

In [ ]:
classifier = sentim_analyzer.train(trainer, training_set)

Training classifier


In [ ]:
for key,value in sorted(sentim_analyzer.evaluate(test_set).items()):
    print('{0}: {1}'.format(key, value))